In [12]:
import pandas as pd
import numpy as np
from datetime import datetime

In [13]:
aggregates_data = pd.read_csv("aggregates.csv")

In [ ]:
#find quantity of unique users
aggregates_data.client_id.nunique()

In [14]:
#delete columns with more than 50% null

deleted_columns = list()
columns = list(aggregates_data.columns)
for column in range(len(columns)):
    percent50 = 177963/2
    if aggregates_data[columns[column]].isnull().sum() > percent50:
        deleted_columns.append(columns[column])

aggregates_data = aggregates_data.drop(columns = deleted_columns)
aggregates_data.head()


#transfer date to timestamp
if str(type(aggregates_data.report_date[0])) == "<class 'str'>":
    for i in range(aggregates_data.shape[0]):
        #print(i)
        aggregates_data.report_date[i] = int(datetime.fromisoformat(aggregates_data.report_date[i]).timestamp())
aggregates_data.head()

<ipython-input-14-9b33fd8435cd>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aggregates_data.report_date[i] = int(datetime.fromisoformat(aggregates_data.report_date[i]).timestamp())


,client_id,report_date,cnt_zp_1m,cnt_zp_3m,cnt_zp_6m,cnt_zp_12m,total_num_cards,cnt_active,cnt_cur,first_opened,...,curdel_30plus_bki,curdel_60plus_bank,curdel_60plus_bki,curdel_tech_bank,curdel_tech_bki,bank_1_29_5y_debtor,bank_30_59_5y_debtor,bank_60_89_5y_debtor,bank_90_119_5y_debtor,bank_120plus_5y_debtor
0,4268,1572555600,NaN,NaN,NaN,NaN,1.0,1.0,0.0,5.548,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8951,1572555600,NaN,NaN,NaN,NaN,2.0,0.0,0.0,125.645,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3485,1572555600,NaN,NaN,NaN,NaN,3.0,1.0,0.0,120.806,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8008,1572555600,2.0,6.0,12.0,24.0,2.0,1.0,0.0,14.516,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1615,1572555600,NaN,NaN,NaN,NaN,4.0,1.0,0.0,210.452,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
#sort data with columns: 'client_id', 'report_date'
aggregates_data = aggregates_data.sort_values(['client_id', 'report_date'])
aggregates_data.fillna(0)

In [107]:
test = pd.read_csv("aggregates.csv")

In [11]:
app_potreb_cnt_dict = dict()
for i in aggregates_data.index:
    user_id = aggregates_data.loc[i, 'client_id']
    app_potreb_cnt_dict[user_id] = aggregates_data.loc[i, 'app_potreb_cnt']

NameError: name 'aggregates_data' is not defined

In [111]:
print(len(app_potreb_cnt_dict))
cnt = 0
cnt_credits = 0
for key in app_potreb_cnt_dict:
    #print(type(app_potreb_cnt_dict[key]))
    if str(app_potreb_cnt_dict[key]) != 'nan' and app_potreb_cnt_dict[key] > 0.0:
        cnt += 1;
        cnt_credits += app_potreb_cnt_dict[key]
print(cnt, cnt_credits)

9895
2898 7091.0


In [71]:
table_of_correlations = aggregates_data.corr()

In [72]:
#finding correlations
#we make array 'mean_score' with mean correlations, wich bigger than 80%
processed_data = aggregates_data
processed_data = processed_data.drop(columns = ['client_id', 'report_date'])

cols = list(processed_data.columns)
ser = np.array([[0] * len(cols)])
mean_score = pd.DataFrame(ser, columns = cols)

table_of_correlations = processed_data.corr()
for column in table_of_correlations:
    corelations = list(table_of_correlations[column])
    cnt = 0
    amount = 0.0
    for ind in range(len(corelations)):
        if corelations[ind] < 1 and corelations[ind] > 0.80:
            amount += corelations[ind]
            cnt += 1
    #print(column, " :   cnt: ", cnt, "  sum: ", amount)
    if cnt != 0:
        mean_score.loc[0, column] = amount/cnt



In [16]:
mean_score.plot()

NameError: name 'mean_score' is not defined

In [75]:

def get_number_or_name_of_max(df, fl = 0):
    # fl == 0 returns number of maximum
    # another fl returns name of maximum
    
    maxi = -1.0
    s = str()
    for column in df:
        elem = df.loc[0, column]
        if elem > maxi:
            maxi = elem
            s = column
    #print("maxi_res: ", maxi)
    if fl == 0:
        return maxi
    else:
        return s

In [76]:
# make 'set_of_deleted_elements' - list with deleting elements
# and a list 'selected_correlating_items' - set with not deleted elements which are responsible of deleted
set_of_deleted_elements = set()
selected_correlating_items = list()
while get_number_or_name_of_max(mean_score, 0) > 0.8:
    name = get_number_or_name_of_max(mean_score, 1)
    selected_correlating_items.append(name)
    ser = table_of_correlations[name]
    corr_list = list(ser)
    for i in range(len(corr_list)):
        if corr_list[i] > 0.8 and corr_list[i] < 1:
            set_of_deleted_elements.add(ser.index[i])
        
    
    mean_score = mean_score.drop(columns = [name])
    
    #delete correlated columns
    for name_to_del in set_of_deleted_elements:
        if name_to_del in mean_score.columns:
            mean_score = mean_score.drop(columns = [name_to_del])

In [79]:
len(set_of_deleted_elements)

27

In [78]:
selected_correlating_items

['total_liab_sum_bki_adv_agr',
 'curdel_30plus_bki',
 'total_liab_sum_bank_adv_agr',
 'max_liab_sum_bki_open',
 'total_num_cards',
 'cnt_zp_6m',
 'outstanding_open',
 'avg_delinquent_debt_total',
 'cnt_adv_repayment_gr_100k',
 'avg_liab_sum_total_open_agr',
 'cnt_adv25_closed',
 'total_curr_payment_rkk',
 'cl_closed_gr50k',
 'bank_count_first_12m_90',
 'bank_count_first_6m_1',
 'cl_closed',
 'cnt_zp_1m',
 'cnt_opened',
 'cnt_opened_6m',
 'cnt_adv_repayment_ls_50k',
 'bank_count_first_12m_30',
 'bank_count_first_6m_60']

In [86]:
aggregates_data.drop(columns = list(set_of_deleted_elements))

,client_id,report_date,cnt_zp_1m,cnt_zp_6m,total_num_cards,cnt_active,cnt_cur,first_opened,last_opened,last6m_opened,...,curdel_30plus_bki,curdel_60plus_bank,curdel_tech_bank,curdel_tech_bki,bank_1_29_5y_debtor,bank_30_59_5y_debtor,bank_60_89_5y_debtor,bank_90_119_5y_debtor,bank_120plus_5y_debtor,mean_corr
0,4268,1572555600,NaN,NaN,1.0,1.0,0.0,5.548,5.548,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,8951,1572555600,NaN,NaN,2.0,0.0,0.0,125.645,118.516,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,3485,1572555600,NaN,NaN,3.0,1.0,0.0,120.806,2.129,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,8008,1572555600,2.0,12.0,2.0,1.0,0.0,14.516,0.032,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,1615,1572555600,NaN,NaN,4.0,1.0,0.0,210.452,20.355,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177958,4351,1580504400,1.0,4.0,2.0,2.0,0.0,154.742,1.548,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
177959,7780,1580504400,4.0,17.0,6.0,1.0,0.0,108.645,12.742,0.0,...,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0
177960,50,1580504400,2.0,13.0,7.0,2.0,0.0,112.355,17.806,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
177961,7928,1580504400,NaN,NaN,2.0,1.0,0.0,11.581,5.968,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [85]:
aggregates_data.sort_values('report_date')

,client_id,report_date,cnt_zp_1m,cnt_zp_3m,cnt_zp_6m,cnt_zp_12m,total_num_cards,cnt_active,cnt_cur,first_opened,...,curdel_60plus_bank,curdel_60plus_bki,curdel_tech_bank,curdel_tech_bki,bank_1_29_5y_debtor,bank_30_59_5y_debtor,bank_60_89_5y_debtor,bank_90_119_5y_debtor,bank_120plus_5y_debtor,mean_corr
86497,100,1546290000,2.0,6.0,12.0,24.0,4.0,1.0,0.0,93.129,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
71685,733,1546290000,NaN,NaN,NaN,NaN,1.0,0.0,0.0,145.355,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
71686,8584,1546290000,3.0,4.0,8.0,14.0,2.0,1.0,0.0,76.032,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
71687,767,1546290000,2.0,6.0,13.0,24.0,1.0,1.0,0.0,12.806,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
71688,5900,1546290000,4.0,8.0,12.0,27.0,4.0,1.0,0.0,91.677,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111443,7058,1601499600,2.0,4.0,10.0,10.0,2.0,2.0,0.0,6.871,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
111442,2427,1601499600,2.0,4.0,15.0,28.0,3.0,1.0,0.0,48.161,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
111441,2137,1601499600,0.0,0.0,0.0,0.0,1.0,1.0,0.0,18.903,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
111449,9641,1601499600,3.0,8.0,16.0,36.0,7.0,2.0,0.0,51.097,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [116]:
aggregates_data.groupby("client_id").sort_value("report_date")

AttributeError: 'DataFrameGroupBy' object has no attribute 'sort_value'

In [5]:
import pandas as pd
import numpy as np
Data = pd.DataFrame()

for i in range(10):
    cols = {'yes' : [1], 'no' : [2]}
    string_in_data = pd.DataFrame(cols)
    Data = Data.append(string_in_data)

In [10]:
Data.reset_index().drop(columns = ['index'])

,yes,no
0,1,2
1,1,2
2,1,2
3,1,2
4,1,2
5,1,2
6,1,2
7,1,2
8,1,2
9,1,2


In [ ]:

aggregates_data.to_csv("sorted_aggregates.csv")